# Edgelist Generator
## This script creates and edge list based on friendship inputs and changes all names to one alter denoted by 'alias_1'.
### Created by: Karina Lopez
### Last Updated: 05/01/2020

In [11]:
import pandas as pd
import numpy as np
import glob
import os
import csv

#setting pandas display options
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


# Enter the csv files you wish to generate an edge list for

In [12]:
# NAME GENERATOR CSV| Must be in the format: dorm_wing, alias_1, dorm_friend#, outside_friend#
network_single_df = pd.read_csv('SN1_NG3.csv')

# ALIASES CSV FILE CONTAINING ALL NICKNAMES/MISSPELLINGS
names_df = pd.read_csv('south_alias_friendship.csv')
alias_headers_wing = list(names_df.columns)
alias_headers_wing = alias_headers_wing[1:]

#ENTER THE WING FOR THE EDGELIST
wing = 'south'

all_friend_headers = list(network_single_df.columns)
all_friend_headers  = all_friend_headers[2:-1]

#ENTER FILE OUTPUT NAME
new_file_name = 'NG3_edgelist_south.csv'

# Run the entire script

In [13]:
#Filter based on wing
network_wing_df = network_single_df[(network_single_df.dorm_wing == wing)]

#Lowercases everything
for header in all_friend_headers:          
        network_wing_df[header] = network_wing_df[header].str.lower()
        
network_wing_df = network_wing_df.reset_index(drop = True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
wing_edgelist_dorm_df = pd.DataFrame()
final_wing_dorm_df = pd.DataFrame()

#for loop for each dorm header
for friend_column in all_friend_headers:
    wing_edgelist_dorm_df = pd.DataFrame()
    list_connections = []

    #for loop for each ego and alter for that column
    for column in network_wing_df[['alias_1', str(friend_column)]]:
        #create a series object
        columnSeriesObj = network_wing_df[column]       
        
        #add alter and ego series to dataframe
        wing_edgelist_dorm_df = wing_edgelist_dorm_df.append(columnSeriesObj)
    
    #create column holding repeating dorm/outside
    list_connections.append(friend_column[0:4])
    list_connections = list_connections * wing_edgelist_dorm_df.shape[1]
    wing_edgelist_dorm_df.loc[len(wing_edgelist_dorm_df), :] = list_connections

    #transpose dataframe and add to final edgelist
    wing_edgelist_dorm_df = wing_edgelist_dorm_df.transpose()
    wing_edgelist_dorm_df.columns = ['ego', 'alter', 'connection']
    final_wing_dorm_df = final_wing_dorm_df.append(wing_edgelist_dorm_df)


# drop all connections with NA in alter and replace outs with outside
final_wing_dorm_df = final_wing_dorm_df.dropna()
final_wing_dorm_df['connection'] = np.where(final_wing_dorm_df['connection'] == 'outs', 'outside', final_wing_dorm_df['connection'])

# reset the index
final_wing_dorm_df = final_wing_dorm_df.reset_index(drop = True)

In [15]:
# remove people without nicknames
names_df = names_df[names_df['alias_2'].notna()]

#reset index
names_df = names_df.reset_index(drop = True)

In [16]:
#new dataframe holding only dorm or outside connections
dorm_connections_wing = final_wing_dorm_df[(final_wing_dorm_df.connection == 'dorm')]
dorm_connections_wing = dorm_connections_wing.reset_index(drop = True)

outside_connections_wing = final_wing_dorm_df[(final_wing_dorm_df.connection == 'outside')]
outside_connections_wing = outside_connections_wing.reset_index(drop = True)

edge_list_headers = ['ego', 'alter']


#Need to cycle through every column of the edgelist
for column_edge in edge_list_headers:
    #need to cycle through each alias
    for aliases in alias_headers_wing:       
        i = 0        
        for i in range(names_df.shape[0]):
            dorm_connections_wing[column_edge] = np.where(dorm_connections_wing[column_edge] == names_df.at[i, aliases], names_df.at[i, 'alias_1'], dorm_connections_wing[column_edge])
            i += 1

#bring back together the dataframe
edge_list_wing = dorm_connections_wing.append(outside_connections_wing)
edge_list_wing = edge_list_wing.reset_index(drop = True)

In [1]:
#Create a csv with all the connections
edge_list_wing.to_csv(new_file_name)

In [18]:
print("Done!")

Done!
